In [1]:
!pip install langchain==0.3.14
!pip install langchain-openai==0.3.0
!pip install langchain-community==0.3.14
!pip install langchain-huggingface==0.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 37.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.39
    Uninstalling langsmith-0.3.39:
      Successfully uninstalled langsmith-0.3.39
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.24
    Uninstalling langchain-0.3.24:
      Successfully uninstalled langchain-0.3.24
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy

In [2]:
!pip install tavily-python

In [2]:
# Tavily client

from tavily import TavilyClient
from langchain.schema import Document
client = TavilyClient(api_key="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")
urls = [
    "https://www.itcportal.com/about-itc/shareholder-value/annual-reports/itc-annual-report-2023/pdf/ITC-Report-and-Accounts-2023.pdf",
    "https://www.itcportal.com/about-itc/shareholder-value/annual-reports/itc-annual-report-2024/pdf/ITC-Report-and-Accounts-2024.pdf"
]
response = client.extract(urls=urls, include_images=False, extract_depth="advanced")

# Process documents
docs = []
for r in response["results"]:
    text = r.get("raw_content", "")
    if text:
        docs.append(Document(page_content=text, metadata={"source": r.get("url")}))

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 50)

In [11]:
chunk = text_splitter.split_documents(docs)

In [12]:
len(docs)

2

In [13]:
len(chunk)

11561

In [14]:
from langchain_openai import OpenAIEmbeddings

In [6]:
f = open("/content/openai_api_key.txt")
api_key = f.read()
embedding_model = OpenAIEmbeddings(openai_api_key = api_key)

In [1]:
!pip install langchain_chroma

In [3]:
from langchain_chroma import Chroma

In [7]:
db = Chroma(collection_name = "vector_database",
            embedding_function= embedding_model,
            persist_directory= "./chroma_db_")

In [15]:
db.add_documents(chunk)

['9f88651b-31d1-4cf6-a4cd-076aa895f7d7',
 '2d9fee52-2bca-481c-b17e-83302a2d825a',
 'c79e1ab9-c542-42bc-be81-63e3e012b7bc',
 '41882da9-3b2d-4105-9f98-6e7853e34e04',
 '279d55a3-37f7-4541-9f85-5df94417fff5',
 '2e024482-f8ae-4b97-87da-6ee6f8278140',
 '64ef5261-7080-4fbb-9f6f-156ccfe3e1df',
 '1b4eb6db-b76e-4e3e-bb65-9ef2692bc51d',
 '12f92160-eaf6-4714-af13-6a1aaf697fd7',
 '4fc902d1-13ec-4979-bcc7-b46c266042ff',
 'ea954951-c45a-40ad-893e-1c4fa5a6486b',
 '23b8cdba-5115-4e77-8dfc-4f21e3088683',
 '367b2755-503d-4b12-bc07-3b5709425e0d',
 'd80ffe58-09a1-4b9f-97b3-e7817d113fe6',
 '43a15926-e0d1-425f-89ea-6a26fecbff4d',
 '3b8bf3f2-59bd-4aaa-9441-4c31a6b4a167',
 'e80c9c6e-1402-49ce-bc16-1ee375d47297',
 'e6b47ba1-0768-49dd-bcef-12314da901aa',
 '65193933-4493-4bab-8a00-321b4b2a4d97',
 '1fbf522f-0275-44c9-a704-1ca82a6a6d04',
 '2c5b6a22-2c53-487c-b9d9-c3b10d10ef79',
 '779dda49-b170-474b-b967-f4ef3fff6453',
 '45e9748d-9e10-4d89-a857-c697e8ff08f4',
 '0acc8d9b-11ee-4b2b-a198-2ed374ba0f1d',
 '1d5aaff6-3d85-

In [ ]:
print(len(db.get()["ids"]))

21


# Step 2 Create a Retrival

In [16]:
query = "what is the revenue if ITC's in 2024 ?"

In [17]:
docs_chroma = db.similarity_search_with_score(query, k =3)

In [18]:
len(docs_chroma)

3

In [19]:
context_text = "\n\n".join ([doc.page_content for doc, scoore in docs_chroma])

In [20]:
print(context_text)

REPORT AND ACCOUNTS 2024 ITC Limited Notes to the Standalone Financial Statements As at 31st March, 2024 (` in Crores) As at 31st March, 2023 (` in Crores) 20. Current Tax Liabilities (Net) Current taxation (net of advance payment) 760.00 776.13 TOTAL 760.00 776.13 * Net of sales returns, damaged

2024 89 ITC Limited ` 3363.06 crores), clocking a resilient growth of 12.5% driven by its expanded global presence and the increasing traction in the company’s strategic accounts. Profit Before Tax stood at ` 628.61 crores (previous year ` 529.66 crores) and Net Profit stood at ` 463.13 crores

REPORT AND ACCOUNTS 2024 319 ITC Limited  (` in Crores) 2024 2023 Capital expenditure Depreciation and amortization Capital expenditure Depreciation and amortization FMCG - Cigarettes 544.32 291.59 185.07 303.74 FMCG - Others 675.67 567.89 418.77 588.03 FMCG - Total 1219.99 859.48 603.84 891.77


In [21]:
for i in docs_chroma:
  print(i[0].page_content)
  print("*" * 50)

REPORT AND ACCOUNTS 2024 ITC Limited Notes to the Standalone Financial Statements As at 31st March, 2024 (` in Crores) As at 31st March, 2023 (` in Crores) 20. Current Tax Liabilities (Net) Current taxation (net of advance payment) 760.00 776.13 TOTAL 760.00 776.13 * Net of sales returns, damaged
**************************************************
2024 89 ITC Limited ` 3363.06 crores), clocking a resilient growth of 12.5% driven by its expanded global presence and the increasing traction in the company’s strategic accounts. Profit Before Tax stood at ` 628.61 crores (previous year ` 529.66 crores) and Net Profit stood at ` 463.13 crores
**************************************************
REPORT AND ACCOUNTS 2024 319 ITC Limited  (` in Crores) 2024 2023 Capital expenditure Depreciation and amortization Capital expenditure Depreciation and amortization FMCG - Cigarettes 544.32 291.59 185.07 303.74 FMCG - Others 675.67 567.89 418.77 588.03 FMCG - Total 1219.99 859.48 603.84 891.77
********

# Step 3 Generation

In [22]:
from langchain_core.prompts import ChatPromptTemplate

In [23]:
prompt_template = '''
              answer the question based only on the following context:
              {context}
              answer the question based on the above context: {question}.
              provide a detailed answer.
              Don't justify your answers'''

In [24]:
prompt_template = ChatPromptTemplate.from_template(prompt_template)

In [25]:
prompt_template

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\n              answer the question based only on the following context:\n              {context}\n              answer the question based on the above context: {question}.\n              provide a detailed answer.\n              Don't justify your answers"), additional_kwargs={})])

In [26]:
from langchain_openai import ChatOpenAI

In [27]:
model = ChatOpenAI(openai_api_key = api_key)

In [28]:
from langchain_core.output_parsers import StrOutputParser

In [29]:
output_parsers = StrOutputParser()

In [30]:
chain = prompt_template | model | output_parsers

In [31]:
input_info = {"context": context_text, "question": "what is the revenue of ITC's in 2024"}

In [32]:
print(chain.invoke(input_info))

The revenue of ITC Limited in 2024 is not explicitly mentioned in the provided context. However, based on the information given, we can calculate the revenue by adding the Profit Before Tax to the Net Profit. 

Profit Before Tax (PBT) for 2024 is mentioned as `628.61 crores, and Net Profit for 2024 is mentioned as `463.13 crores. 

Therefore, Revenue for 2024 can be calculated as:

Revenue = Profit Before Tax + Net Profit
Revenue = `628.61 crores + `463.13 crores
Revenue = `1091.74 crores

Hence, the revenue of ITC Limited in 2024 is `1091.74 crores.
